In [1]:
from copyreg import constructor
from evogym import is_connected, has_actuator, get_full_connectivity, hashable
from evogym.envs import StepsUp, WalkingFlat
import struct
from typing import List, Tuple
import numpy as np
import dill
from graph import Graph
from evolution_strategies import tournament_selection
from population import Population
from arg_parser import parse_args
from pygifsicle import optimize
import imageio
import test

In [2]:
FILE_PATH = "results_ladder/"
GENERATION = "102"
ENV = "StepsUp" # "[WalkingFlat, StepsUp]"

In [3]:
with open(f"{FILE_PATH}structure_{GENERATION}.pkl", "rb") as file:
    structure_pop = dill.load(file)
    
with open(f"{FILE_PATH}controler_{GENERATION}.pkl", "rb") as file:
    controller_pop = dill.load(file)
    
def get_observation(env):
    a = env.get_vel_com_obs("robot")
    b = env.get_pos_com_obs("robot")
    c = env.get_floor_obs("robot", ["ground"], 5)
    return np.concatenate((a, b, c))

# def get_observation(env):
#     a = env.get_vel_com_obs("robot")
#     b = env.get_pos_com_obs("robot")
#     return np.concatenate((a, b))
    
def generate_robot(g: Graph, structure):
    robot = np.zeros(structure)
    for i in range(structure[0]):
        for j in range(structure[1]):
            input = (i - (structure[0] // 2),
                     j - (structure[1] // 2))
            graph_out = g.operate(input)
            node = np.argmax(graph_out)
            robot[i][j] = node
    return robot

def get_best_controller(robot, controller_population, n_steps):
    robot = generate_robot(design, (5,5))    
    connections = get_full_connectivity(robot)

    
    env = StepsUp(body=robot, connections=connections)
    
    fitness = []
    cont = 0
    for controller in controller_population.indvs:
        env.reset()
        reward = test.calculate_reward(env, controller, 500)
        fitness.append((cont, reward, controller.id))
        cont += 1
        
    fitness.sort(key=lambda x: (x[1], x[0]))
    
    return fitness

In [4]:
structure_pop.indvs.sort(key=lambda x: (x.original_fit*-1, x.id))
controller_pop.indvs.sort(key=lambda x: (x.original_fit*-1, x.id))
cont = 0
print("\t ID \t  Fitness \t Species")
for design in structure_pop.indvs:
    if design.original_fit > -999:
       
        print(f"{cont}:\t {design.id}\t| {design.original_fit:.4f}\t| {design.species_id}")
    cont+=1

	 ID 	  Fitness 	 Species
0:	 2339	| 3.2880	| 549
1:	 16488	| 2.7900	| 4
2:	 16644	| 2.3824	| 582
3:	 16493	| 1.4556	| 4
4:	 15333	| 1.4079	| 4
5:	 16646	| 1.3894	| 4
6:	 16615	| 0.9289	| 580
7:	 16621	| 0.8262	| 4
8:	 16624	| 0.8066	| 4
9:	 16628	| 0.5771	| 4
10:	 16622	| 0.4411	| 581
11:	 16663	| 0.2682	| 584
12:	 16648	| 0.2533	| 4
13:	 16634	| 0.2182	| 4
14:	 16665	| 0.0848	| 549
15:	 16661	| 0.0355	| 581
16:	 16650	| 0.0292	| 583
17:	 16636	| 0.0130	| 4
18:	 16659	| -0.0066	| 4
19:	 16626	| -0.0708	| 4
20:	 16655	| -0.0862	| 4


In [5]:
STEPS = 600
CONTROLLER_ID = 0
DESIGN_ID = 2
SAVE_GIF = False

In [6]:
controller = controller_pop.indvs[CONTROLLER_ID]
print(controller.id)
design =structure_pop.indvs[DESIGN_ID]

robot = generate_robot(design, (5,5))
print(robot)
connections = get_full_connectivity(robot)

if ENV == "StepsUp":
    env = StepsUp(body=robot, connections=connections)
else:
    env = WalkingFlat(body=robot, connections=connections)



env.reset()
if not SAVE_GIF:
    env.render('screen')

reward = 0
actuators = env.get_actuator_indices("robot")
imgs = []
for _ in range(STEPS):
    obs = get_observation(env)

    action_by_actuator = controller.operate(obs, reset_fit=False)
    action = [action_by_actuator[i] for i in actuators]
#     action = np.clip(action, .6, 1.6)
    _, r, done, _ = env.step(np.array(action))
    
    if SAVE_GIF:
        imgs.append(env.render(mode='img'))
    else:
        env.render('screen')
    reward += r

    if done:
        print("AAA")
        break

        
print(f'\ntotal reward: {round(reward, 5)}\n')
env.close()

if SAVE_GIF:
    print(len(imgs))
    imageio.mimsave(f'{ENV}_gen_{GENERATION}_c{CONTROLLER_ID}_d_{DESIGN_ID}.gif', imgs, duration=(1/60))

12262
[[3. 3. 1. 1. 2.]
 [3. 3. 1. 1. 2.]
 [3. 3. 1. 1. 2.]
 [3. 3. 1. 1. 2.]
 [3. 3. 1. 1. 2.]]

total reward: -0.01294



In [7]:
if SAVE_GIF:
    optimize(f'{ENV}_gen_{GENERATION}_c{CONTROLLER_ID}_d_{DESIGN_ID}.gif')

In [8]:
aux = get_best_controller(robot, controller_pop, 600)
print(aux)

[(2, -0.04458483797421986, 14673), (8, -0.0385576214406223, 16735), (3, -0.031149284260777454, 13274), (18, -0.029192391309832533, 16733), (14, -0.02673170125021962, 16710), (22, -0.021988592973069654, 16701), (16, -0.01619172421008247, 16719), (11, -0.014463164860610278, 16723), (0, -0.013791273941857973, 12262), (19, -0.013679612027845567, 16727), (4, -0.013549792689436346, 16687), (26, -0.011713565566719486, 16737), (27, -0.011299915235047364, 16725), (23, -0.009139255104605615, 16731), (17, -0.005878975937403053, 16714), (10, -0.004934934230262311, 16716), (29, -0.004115824286682557, 16689), (25, -0.0037141487676223273, 16694), (30, -0.0001820185549037956, 16697), (1, 7.879132742122374e-05, 10660), (5, 0.0005918257624430989, 16729), (31, 0.0014986744153973053, 16704), (21, 0.002932679151569473, 16699), (28, 0.00343437716398437, 16706), (9, 0.005061945348423191, 16708), (7, 0.0053311291419561035, 16696), (24, 0.005596095737637252, 16712), (6, 0.009809542442853425, 16703), (13, 0.011